# Модель, прогнозирующая успешность прохождения обучения студентов

### Цели и задачи проекта

Цель: используя данные о первых двух днях на онлайн-курсе, предсказать, наберет ли пользователь 40 и более баллов. Качество модели оценить по метрике ROC-AUC.

### Описание данных

`events_train.csv` - данные о действиях, которые совершают студенты со стэпами

- `step_id` - id стэпа
- `user_id` - анонимизированный id юзера
- `timestamp` - время наступления события в формате unix date
- `action` - событие, возможные значения: 
    - `discovered` - пользователь перешел на стэп
    - `viewed` - просмотр шага,
    - `started_attempt` - начало попытки решить шаг, ранее нужно было явно нажать на кнопку - начать решение, перед тем как приступить к решению практического шага
    - `passed` - удачное решение практического шага


`submissions_train.csv` - данные о времени и статусах сабмитов к практическим заданиям

- `step_id` - id стэпа
- `timestamp` - время отправки решения в формате unix date
- `submission_status` - статус решения
- `user_id` - анонимизированный id юзера

### Содержимое проекта

1. Загрузка данных и знакомство с ними.
2. Предобработка данных и подготовка их к исследованию.
3. Формирование и тестирование моделей.
4. Общий вывод и рекомендации.

---

## 1. Загрузка данных и знакомство с ними

In [1]:
# Библиотеки для обработки и анализа данных
import pandas as pd
import numpy as np

# Cтатистический анализ
import scipy.stats as stats

# Библиотека для работы со временем  
from time import time         

# Библиотеки машинного обучения
from sklearn.model_selection import RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier                               
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Глобальные настройки
pd.options.mode.copy_on_write = True
RANDOM_STATE = 9
CV_FOLDS = 5

In [2]:
# Загружаем данные
events_df = pd.read_csv('...')
subm_df = pd.read_csv('...')

In [3]:
# Создаем словарь датафреймов
dic_df = {
    'events_df': events_df,
    'subm_df': subm_df
}

# Выводим информацию о датасетах и первые строки.
for name, dfs in dic_df.items():
    print('=' * 50)
    print(f'Информация о датасете {name}')
    print('=' * 50, '\n')
    print(dfs.info(), '\n')
    display(dfs.head())

Информация о датасете events_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3480703 entries, 0 to 3480702
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   step_id    int64 
 1   timestamp  int64 
 2   action     object
 3   user_id    int64 
dtypes: int64(3), object(1)
memory usage: 106.2+ MB
None 



,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632


Информация о датасете subm_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509104 entries, 0 to 509103
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   step_id            509104 non-null  int64 
 1   timestamp          509104 non-null  int64 
 2   submission_status  509104 non-null  object
 3   user_id            509104 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 15.5+ MB
None 



,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [4]:
# Подсчитываем процент строк с пропусками
for name, dfs in dic_df.items():
    print('=' * 73)    
    print(f'Анализ пропущенных значений в {name}')
    print('=' * 73) 
    
    missing_df = pd.DataFrame({
        'Без пропусков': dfs.notna().sum(),
        'Кол-во пропусков': dfs.isna().sum(),
        'Процент пропусков (%)': round(dfs.isna().mean() * 100, 1)
    })
    
    print(missing_df, '\n')

Анализ пропущенных значений в events_df
           Без пропусков  Кол-во пропусков  Процент пропусков (%)
step_id          3480703                 0                    0.0
timestamp        3480703                 0                    0.0
action           3480703                 0                    0.0
user_id          3480703                 0                    0.0 

Анализ пропущенных значений в subm_df
                   Без пропусков  Кол-во пропусков  Процент пропусков (%)
step_id                   509104                 0                    0.0
timestamp                 509104                 0                    0.0
submission_status         509104                 0                    0.0
user_id                   509104                 0                    0.0 



Данные загружены, пропусков нет.

---

## 2. Предобработка данных и подготовка их к исследованию

- Проверим на дубликаты и удалим при наличии.

In [5]:
print(f'Количество полных дубликатов в events_df: {events_df.duplicated().sum()} строк.')

nodup_events_df = events_df.drop_duplicates().reset_index(drop=True)
print(f'Процент удаленных строк events_df: {round((1 - len(nodup_events_df) / len(events_df)) * 100, 2)}%')

Количество полных дубликатов в events_df: 2333 строк.
Процент удаленных строк events_df: 0.07%


In [6]:
print(f'Количество полных дубликатов subm_df: {subm_df.duplicated().sum()} строк.')

nodup_subm_df = subm_df.drop_duplicates().reset_index(drop=True)
print(f'Процент удаленных строк в subm_df: {round((1 - len(nodup_subm_df) / len(subm_df)) * 100, 2)}%')

Количество полных дубликатов subm_df: 107 строк.
Процент удаленных строк в subm_df: 0.02%


- Преобразование типов данных.

In [7]:
nodup_events_df['date'] = pd.to_datetime(nodup_events_df['timestamp'], unit='s')
nodup_subm_df['date'] = pd.to_datetime(nodup_subm_df['timestamp'], unit='s')

nodup_events_df['day'] = pd.to_datetime(nodup_events_df['timestamp'], unit='s').dt.date
nodup_subm_df['day'] = pd.to_datetime(nodup_subm_df['timestamp'], unit='s').dt.date

Сформируем таблицу с пользователями и данными о том набрали они 40 и более баллов или нет.

In [8]:
total_df = (
    nodup_subm_df[nodup_subm_df['submission_status']=='correct']
        .groupby('user_id')
        .agg(more_40=('step_id', 'nunique')
            )
        .reset_index()
)

total_df['more_40'] = np.where(total_df['more_40']>=40, 1, 0)

Отфильтруем данные в таблице с событиями за первые два дня.

In [9]:
# Два дня в секундах
period = (60 * 60 * 24) * 2

# Фильтруем события за первые два дня в nodup_events_df
nodup_events_df['first_date'] = nodup_events_df.groupby('user_id')['date'].transform('min')
events_2d_df = nodup_events_df[(nodup_events_df['date'] - nodup_events_df['first_date']).dt.total_seconds()<=period]

# Фильтруем события за первые два дня в nodup_subm_df
nodup_subm_df['first_date'] = nodup_subm_df.groupby('user_id')['date'].transform('min')
subm_2d_df = nodup_subm_df[(nodup_subm_df['date'] - nodup_subm_df['first_date']).dt.total_seconds()<=period]

Формируем признаки для обучения модели из таблицы событий по первым двум дням активности.

In [10]:
feature_df = pd.DataFrame()
feature_df = (
    pd.get_dummies(events_2d_df, columns=['action'], dtype=int)
        .groupby('user_id')
        .agg(
            nuniq_steps_events=('step_id', 'nunique'),
            cnt_discovered=('action_discovered', 'sum'),
            cnt_passed=('action_passed', 'sum'),
            cnt_started_attempt=('action_started_attempt', 'sum'),
            cnt_viewed=('action_viewed', 'sum'),
            nuniq_days_events=('day', 'nunique')
        )
        .reset_index()
)

In [11]:
# Добавляем таблицу признаков в датасет для обучения
total_df = total_df.merge(feature_df, on='user_id', how='outer')

Сформируем таблицу с признаками из таблицы прохождения тестов.

In [12]:
subm_2d_df['last_step'] = subm_2d_df.groupby('user_id')['step_id'].transform('max')
subm_2d_df['first_step'] = subm_2d_df.groupby('user_id')['step_id'].transform('min')

temp_df = pd.get_dummies(subm_2d_df, columns=['submission_status'], dtype=int)

# Формируем таблицу признаков
feature_df = (
    temp_df[temp_df['step_id']==temp_df['last_step']]
        .groupby('user_id')
        .agg(            
            correct_on_last_step=('submission_status_correct', 'sum'),
            wrong_on_last_step=('submission_status_wrong', 'sum')
        )
        .reset_index()
)

# Добавляем таблицу признаков в датасет для обучения
total_df = total_df.merge(feature_df, on='user_id', how='outer').fillna(0)                

In [13]:
# Формируем таблицу признаков
feature_df = (
    temp_df[temp_df['step_id']==temp_df['first_step']]
        .groupby('user_id')
        .agg(
            correct_on_first_step=('submission_status_correct', 'sum'),
            wrong_on_first_step=('submission_status_wrong', 'sum')
            )
        .reset_index()
)

# Добавляем таблицу признаков в датасет для обучения
total_df = total_df.merge(feature_df, on='user_id', how='outer').fillna(0)    

In [14]:
# Формируем таблицу признаков
feature_df = (
    pd.get_dummies(subm_2d_df, columns=['submission_status'], dtype=int)
        .groupby('user_id')
        .agg(
            nuniq_step_subm=('step_id', 'nunique'),
            nuniq_correct_step=('submission_status_correct', 'nunique'),
            cnt_correct=('submission_status_correct', 'sum'),
            cnt_wrong=('submission_status_wrong', 'sum'),
            nuniq_days_subm=('day', 'nunique')
        )
        .reset_index()
)

# Добавляем таблицу признаков в датасет для обучения
total_df = total_df.merge(feature_df, on='user_id', how='outer').fillna(0)

In [15]:
print("Распределение целевой переменной:")
print(
    total_df['more_40'].value_counts()
        .reset_index()
        .astype({'more_40': 'int'})
        .to_string(index=False)
)
print(f"\nДоля пользователей, набравших больше 40 баллов: {total_df['more_40'].mean():.3f}")

Распределение целевой переменной:
 more_40  count
       0  17269
       1   1965

Доля пользователей, набравших больше 40 баллов: 0.102


In [16]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19234 entries, 0 to 19233
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   user_id                19234 non-null  int64  
 1   more_40                19234 non-null  float64
 2   nuniq_steps_events     19234 non-null  int64  
 3   cnt_discovered         19234 non-null  int32  
 4   cnt_passed             19234 non-null  int32  
 5   cnt_started_attempt    19234 non-null  int32  
 6   cnt_viewed             19234 non-null  int32  
 7   nuniq_days_events      19234 non-null  int64  
 8   correct_on_last_step   19234 non-null  float64
 9   wrong_on_last_step     19234 non-null  float64
 10  correct_on_first_step  19234 non-null  float64
 11  wrong_on_first_step    19234 non-null  float64
 12  nuniq_step_subm        19234 non-null  float64
 13  nuniq_correct_step     19234 non-null  float64
 14  cnt_correct            19234 non-null  float64
 15  cn

После проделанной работы получили датафрейм со следующими признаками для машинного обучения:
- nuniq_steps_events - количество уникальных стэпов с событиями,
- cnt_discovered - количество переходов на стэп,
- cnt_passed - количество удачных решений стэпа,
- cnt_started_attempt - количество попыток решить стэп,
- cnt_viewed - количество просмотров стэпов,
- nuniq_days_events - число уникальных дней с событиями, 
- correct_on_last_step - число решений со статусом correct на последнем стэпе,
- wrong_on_last_step - число решений со статусом wrong на последнем стэпе,
- correct_on_first_step - число решений со статусом correct на первом стэпе,
- wrong_on_first_step - число решений со статусом wrong на первом стэпе,
- nuniq_step_subm - количество уникальных стэпов со статусами решений,
- nuniq_correct_step - количество степов со статусом решений correct,
- cnt_correct - количество решений со статусом correct,
- cnt_wrong - количество решений со статусом wrong,
- nuniq_days_subm - число уникальных дней с решениями.

---
## 3. Формирование и тестирование моделей

- Подготовка данных для обучения

In [17]:
def split_data(X, y, test_size=0.2):
    """
    Подготовка данных для обучения
    """
    # Разделение на train и test
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=test_size, 
                                                        random_state=RANDOM_STATE,
                                                        stratify=y
                                                       )
        
    print(f"Размеры данных:")
    print(f"Train: {X_train.shape}, \nTest: {X_test.shape}")
    
    return X_train, X_test, y_train, y_test

- Настройка XGBoost

In [18]:
def ml_xgb(X_train, y_train):
    """
    Оптимизация гиперпараметров XGBoost
    """   
    
    # расчет scale_pos_weight
    class_counts = np.bincount(y_train)
    scale_pos_weight = (class_counts[0] / class_counts[1]).round(1)
    
    xgb_params = {
        'n_estimators': [100, 200, 300, 400, 500],
        'max_depth': [2, 3, 4, 5, 8], #
        'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.15],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9],
        'scale_pos_weight': [scale_pos_weight],
        'reg_alpha': [0, 0.1, 0.5, 1],
        'reg_lambda': [1, 2, 3, 5]
    } 

    # Стратифицированное разбиение
    cv_stratified = StratifiedKFold(
        n_splits=CV_FOLDS, 
        shuffle=True, 
        random_state=RANDOM_STATE
    )
    
    model_xgb = XGBClassifier(
        eval_metric='auc',
        random_state=RANDOM_STATE
    )
    
    model_xgb_search = RandomizedSearchCV(
        model_xgb,
        xgb_params,
        n_iter=50, 
        cv=cv_stratified,
        scoring='roc_auc',
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    start_time = time()  
    print("Модель XGBoost обучается...", end="  ")
    model_xgb_search.fit(X_train, y_train)
    print("Ok.")

    duration = time() - start_time
    print(f"Время выполнения: {(duration//60):.0f} мин {(duration%60):.0f} сек")        
    print(f"Лучшие параметры XGBoost: {model_xgb_search.best_params_}\n")
    
    return model_xgb_search.best_estimator_

- Настройка Random Forest

In [19]:
def ml_rfс(X_train, y_train):
    """
    Оптимизация гиперпараметров Random Forest
    """   

    rfс_params = {
         'n_estimators': [100, 200, 300, 400],
         'max_depth': [3, 5, 7, 10],
         'min_samples_split': [10, 15, 20, 30],
         'min_samples_leaf': [5, 10, 15, 20],
         'max_features': ['sqrt', 'log2', 0.3],
         'max_samples': [0.6, 0.7, 0.8, 0.9],
         'criterion': ['gini', 'entropy'],
         'bootstrap': [True],
         'class_weight': ['balanced'],
    }

    # Стратифицированное разбиение
    cv_stratified = StratifiedKFold(
        n_splits=CV_FOLDS, 
        shuffle=True, 
        random_state=RANDOM_STATE
    )
    
    model_rfс = RandomForestClassifier(random_state=RANDOM_STATE)
    
    model_rfс_search = RandomizedSearchCV(
        model_rfс, 
        rfс_params,
        n_iter=70,
        cv=cv_stratified,
        scoring='roc_auc',
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    start_time = time()  
    print("Модель Random Forest обучается...", end="  ")
    model_rfс_search.fit(X_train, y_train)
    print("Ok.")

    duration = time() - start_time
    print(f"Время выполнения: {(duration//60):.0f} мин {(duration%60):.0f} сек")   
    print(f"Лучшие параметры Random Forest: {model_rfс_search.best_params_}\n")
    
    return model_rfс_search.best_estimator_

- Настройка Gradient Boosting

In [20]:
def ml_gbc(X_train, y_train):
    """
    Оптимизация гиперпараметров Gradient Boosting
    """
    
    gbc_params = {
        'n_estimators': [500],
        'max_depth':  [2, 3, 4, 5],
        'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.15],
        'min_samples_split': [20, 50, 70, 100],
        'min_samples_leaf': [10, 20, 30], 
        'subsample': [0.7, 0.8, 0.9],
        'max_features': ['sqrt', 'log2', 0.3]
    }

    # Стратифицированное разбиение
    cv_stratified = StratifiedKFold(
        n_splits=CV_FOLDS, 
        shuffle=True, 
        random_state=RANDOM_STATE
    )
    
    model_gbc = GradientBoostingClassifier(
        n_iter_no_change=30,
        validation_fraction=0.1,
        random_state=RANDOM_STATE
    )
    
    model_gbc_search = RandomizedSearchCV(
        model_gbc, 
        gbc_params,
        n_iter=50,
        cv=cv_stratified,
        scoring='roc_auc',
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    start_time = time()  
    print("Модель Gradient Boosting обучается...", end="  ")
    model_gbc_search.fit(X_train, y_train)
    print("Ok.")
    
    duration = time() - start_time
    print(f"Время выполнения: {(duration//60):.0f} мин {(duration%60):.0f} сек")    
    print(f"Лучшие параметры Gradient Boosting: {model_gbc_search.best_params_}\n")
    
    return model_gbc_search.best_estimator_

- Настройка CatBoost

In [21]:
def ml_catb(X_train, y_train):
    """
    Оптимизация гиперпараметров CatBoost
    """
    
    catb_params = {
        'iterations': [700],
        'depth': [2, 4, 6, 8], 
        'l2_leaf_reg': [1, 2, 3, 5],
        'auto_class_weights': ['Balanced'],
        'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.15]
    }

    # Стратифицированное разбиение
    cv_stratified = StratifiedKFold(
        n_splits=CV_FOLDS, 
        shuffle=True, 
        random_state=RANDOM_STATE
    )
    
    model_catb = CatBoostClassifier(
        loss_function='Logloss',
        early_stopping_rounds=30, 
        eval_metric='AUC',
        verbose=False,
        random_state=RANDOM_STATE,
        thread_count=-1
    )
    
    model_catb_search = RandomizedSearchCV(
        model_catb, 
        catb_params,
        cv=cv_stratified,
        scoring='roc_auc',
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    start_time = time()  
    print("Модель CatBoost обучается...", end="  ")
    model_catb_search.fit(X_train, y_train)
    print("Ok.")

    duration = time() - start_time
    print(f"Время выполнения: {(duration//60):.0f} мин {(duration%60):.0f} сек")   
    print(f"Лучшие параметры CatBoost: {model_catb_search.best_params_}\n")
    
    return model_catb_search.best_estimator_

- Создание ансамбля Stacking

In [22]:
def ml_stacking(list_models, X_train, y_train):
    """
    Создание Stacking
    """

    meta_pipeline = make_pipeline(
        StandardScaler(),
        LogisticRegression(
            C=0.1,
            penalty='l2',
            max_iter=1000,
            random_state=RANDOM_STATE,
            n_jobs=-1
        )
    )
        
    ens_stacking = StackingClassifier(
        estimators=[(name, model) for name, model in list_models.items()],
        final_estimator=meta_pipeline,
        cv=CV_FOLDS,
        passthrough=False,
        n_jobs=-1
    )
    
    start_time = time()  
    print("Ансамбль Stacking Classifier обучается...", end="  ")
    ens_stacking.fit(X_train, y_train)
    print("Ok.")

    duration = time() - start_time
    print(f"Время выполнения: {(duration//60):.0f} мин {(duration%60):.0f} сек\n")    

    print("Состав ансамбля:")
    for name, model in list_models.items():
        print(f"- {type(model).__name__}")
        
    return ens_stacking

- Оценка моделей

In [23]:
def evaluate_model(models, X_train, y_train, cv_n=max(3, CV_FOLDS)):
    """
    Оценка моделей
    """   
    
    # Оценка через кросс-валидацию
    scores_dic = {}

    rskf = RepeatedStratifiedKFold(
        n_splits=cv_n, 
        n_repeats=7, 
        random_state=RANDOM_STATE
    )
    
    for name, model in models.items():
        scores = cross_val_score(
            model, 
            X_train, 
            y_train, 
            cv=rskf, 
            scoring='roc_auc', 
            n_jobs=-1
        )
        
        scores_dic[name] = {'all_scores': scores}
    
    scores_df = pd.DataFrame(scores_dic).T
    
    return scores_df

- Выбор лучшей модели по нижней границе доверительного интервала ROC-AUC

In [24]:
def select_model(df, confidence=0.95):
    """
    Выбирает лучшую модель по нижней границе доверительного интервала ROC-AUC
    """
    
    results = []
    
    for model_name, row in df.iterrows():
        scores = row['all_scores']
        n_folds = len(scores)
        mean_score = np.mean(scores)
        std_score = np.std(scores, ddof=1)
                
        # Расчет доверительного интервала
        t_value = stats.t.ppf((1 + confidence) / 2, n_folds - 1)
        sem = np.std(scores, ddof=1) / np.sqrt(n_folds)
        ci_lower = np.mean(scores) - t_value * sem
        ci_upper = np.mean(scores) + t_value * sem
        
        results.append({
            'model': model_name,
            'mean_roc_auc': mean_score,
            'median': np.median(scores),
            'std_roc_auc': std_score,
            'n_folds': n_folds,
            'ci_lower': ci_lower,
            'ci_upper': ci_upper
        })
    
    # Создаем датафрейм с результатами
    results_df = pd.DataFrame(results)
    results_df.set_index('model', inplace=True)
  
    # Выбираем модель с максимальной нижней границей CI
    best_model_name = results_df['ci_lower'].idxmax()
    best_model_metrics = results_df.loc[best_model_name]
    
    # Сортируем результаты по ci_lower
    results_df = results_df.sort_values('ci_lower', ascending=False)
    results_df['rank'] = results_df['ci_lower'].rank(method='min', ascending=False)

    title_table_models = "СРАВНЕНИЕ ВСЕХ МОДЕЛЕЙ (отсортировано по CI lower):"
    print()
    print(title_table_models)
    print("-" * len(title_table_models))
    print(results_df.to_string(index_names=False), "\n")

    title_best_model = f"ЛУЧШАЯ МОДЕЛЬ: {best_model_name}"
    print(title_best_model)
    print("-" * len(title_best_model))
    print(f"ROC-AUC: {best_model_metrics['mean_roc_auc']:.4f} ± {best_model_metrics['std_roc_auc']:.4f}")
    print(f"Доверительный интервал ({confidence*100:.0f}%): [{best_model_metrics['ci_lower']:.4f}, {best_model_metrics['ci_upper']:.4f}]")

    # Анализ качества модели
    quality = ("ОТЛИЧНОЕ" if best_model_metrics['mean_roc_auc'] > 0.9 else 
                   "ХОРОШЕЕ" if best_model_metrics['mean_roc_auc'] > 0.8 else "ТРЕБУЕТ ДОРАБОТКИ")
    
    print(f"Качество модели: {quality}")
    
    # Анализ стабильности
    cv = best_model_metrics['std_roc_auc'] / best_model_metrics['mean_roc_auc']
    stability_level = ("ОТЛИЧНАЯ" if cv < 0.05 else 
                           "ХОРОШАЯ" if cv < 0.1 else 
                               "УМЕРЕННАЯ" if cv < 0.25 else "НИЗКАЯ")
    
    print(f"Стабильность: {stability_level} (коэф. вариации: {cv:.4f})")
    
    # Дополнительная информация о выборе
    title_justification = "ОБОСНОВАНИЕ ВЫБОРА:"
    print()
    print(title_justification)
    print("-" * len(title_justification))    
    print(f"Модель '{best_model_name}' выбрана, потому что имеет")
    print(f"наивысшую нижнюю границу доверительного интервала ({results_df.loc[best_model_name,'ci_lower']:.4f}),")
    print(f"что означает наибольшую гарантированную производительность с уверенностью {confidence*100:.0f}%.")

    return best_model_name, results_df

- Полный пайплайн обучения и оценки моделей

In [25]:
def full_pipeline(X, y):
    """
    Полный пайплайн обучения и оценки моделей
    """
    # 1. Подготовка данных
    print("=" * 80)
    print("ПОДГОТОВКА ДАННЫХ")
    print("=" * 80)
    
    X_train, X_test, y_train, y_test = split_data(X, y)
    
    # 2. Оптимизация всех моделей
    print("\n" + "=" * 80)
    print("НАСТРОЙКА МОДЕЛЕЙ")
    print("=" * 80)
    
    list_models = {
        'XGBoost': ml_xgb,
        'RandomForest': ml_rfс,
        'GradientBoosting': ml_gbc,
        'CatBoost': ml_catb
    }

    models = {}

    for name, func in list_models.items():
        models[name] = func(X_train, y_train)

    # 3. Создание ансамбля
    print("=" * 80)
    print("СОЗДАНИЕ АНСАМБЛЯ")
    print("=" * 80)
    
    models['Stacking'] = ml_stacking(models, X_train, y_train)
    
    # 4. Оценка всех моделей и выбор лучшей
    print("=" * 80)
    print("РЕЗУЛЬТАТЫ ВЫБОРА МОДЕЛИ ПО НИЖНЕЙ ГРАНИЦЕ ДОВЕРИТЕЛЬНОГО ИНТЕРВАЛА")
    print("=" * 80)

    # 4.1 Оценка через кросс-валидацию
    train_scores_df = evaluate_model(models, X_train, y_train)
    
    # 4.2 Выбираем лучшую модель
    best_model_name, results_df = select_model(train_scores_df)
    best_model = models[best_model_name]   
    
    # 5. Финальная оценка на тесте с лучшей моделью
    print("\n" + "=" * 80)
    print("ФИНАЛЬНАЯ ОЦЕНКА НА ТЕСТЕ")
    print("=" * 80)   
    
    test_roc_auc = roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1])
    
    print(f"Лучшая модель: {best_model_name}")
    print(f"Test ROC-AUC: {test_roc_auc:.4f}")
   
    return {
        'models': models,
        'best_model_name': best_model_name,
        'best_model': best_model,
        'test_roc_auc': test_roc_auc,
        'comparison_models': results_df
    }

In [26]:
# Разделяем датасет на признаки и целевое значение
X = total_df.drop(['user_id', 'more_40'], axis=1)
y = total_df['more_40']

In [27]:
results = full_pipeline(X, y)

ПОДГОТОВКА ДАННЫХ
Размеры данных:
Train: (15387, 15), 
Test: (3847, 15)

НАСТРОЙКА МОДЕЛЕЙ
Модель XGBoost обучается...  Ok.
Время выполнения: 0 мин 35 сек
Лучшие параметры XGBoost: {'subsample': 0.7, 'scale_pos_weight': 8.8, 'reg_lambda': 1, 'reg_alpha': 1, 'n_estimators': 300, 'max_depth': 4, 'learning_rate': 0.01, 'colsample_bytree': 0.9}

Модель Random Forest обучается...  Ok.
Время выполнения: 1 мин 35 сек
Лучшие параметры Random Forest: {'n_estimators': 300, 'min_samples_split': 15, 'min_samples_leaf': 10, 'max_samples': 0.9, 'max_features': 0.3, 'max_depth': 7, 'criterion': 'entropy', 'class_weight': 'balanced', 'bootstrap': True}

Модель Gradient Boosting обучается...  Ok.
Время выполнения: 1 мин 24 сек
Лучшие параметры Gradient Boosting: {'subsample': 0.9, 'n_estimators': 500, 'min_samples_split': 70, 'min_samples_leaf': 30, 'max_features': 'sqrt', 'max_depth': 5, 'learning_rate': 0.005}

Модель CatBoost обучается...  Ok.
Время выполнения: 2 мин 25 сек
Лучшие параметры CatBoost

---

## 4. Общий вывод и рекомендации

##### В данной работе проанализированы данные о решениях и действиях для 19234 студентов за первые два дня прохождения курса (с момента начала первого решения), удалены дубликаты и созданы дополнительные признаки для обучения моделей.

Датасет для анализа содержит 19234 строк (студентов), 15 различных признаков и целевую переменную. Для предсказания наберет студент 40 баллов или нет, мы использовали алгоритмы: XGBClassifier, RandomForestClassifier, GradientBoostingClassifier, CatBoostClassifier и ансамбль Stacking. 

Все модели показали практически одинаковые результаты. Лучшая модель на основе максимальной нижней границы доверительного интервала ROC-AUC - ансамбль Stacking. На тестовой выборке результат ROC-AUC - 0.8915.

Для увеличения метрики качества можно составить дополнительные признаки, попробовать другие алгоритмы или увеличить количество анализируемых дней, по которым строим метрики.